In [1]:
import pandas as pd
import numpy as np

In [2]:
car_blend = pd.read_csv('make_hard_blend.csv')
car_blend

,car_id,target_class,model_1,model_2,model_3,model_4,model_5
0,car_0,NaN,electro_bug,gear_stick,electro_bug,electro_bug,engine_ignition
1,car_1,NaN,engine_check,engine_overheat,electro_bug,engine_overheat,engine_overheat
2,car_2,NaN,another_bug,electro_bug,engine_overheat,engine_overheat,gear_stick
3,car_3,NaN,electro_bug,electro_bug,electro_bug,wheel_shake,electro_bug
4,car_4,NaN,engine_ignition,another_bug,engine_check,engine_check,engine_check
...,...,...,...,...,...,...,...
1908,car_1908,NaN,break_bug,break_bug,engine_overheat,break_bug,another_bug
1909,car_1909,NaN,break_bug,break_bug,wheel_shake,break_bug,gear_stick
1910,car_1910,NaN,engine_overheat,engine_overheat,engine_overheat,engine_overheat,break_bug
1911,car_1911,NaN,engine_check,engine_check,engine_check,electro_bug,break_bug


In [20]:
car_blend['target_class'] = car_blend.drop(columns=['car_id','target_class']).apply(lambda x: x.mode(), axis=1)[0]

In [22]:
car_blend[['car_id', 'target_class']].to_csv('submission_car_blend.csv', index=False)

In [28]:
cat_preds = pd.read_csv('catboost_preds.csv')

In [30]:
lgbm_preds = pd.read_csv('lgbm_preds.csv')
xgb_preds = pd.read_csv('xgb_preds.csv')

In [31]:
tot_preds = cat_preds + lgbm_preds + xgb_preds
tot_preds

,another_bug_proba,break_bug_proba,electro_bug_proba,engine_check_proba,engine_fuel_proba,engine_ignition_proba,engine_overheat_proba,gear_stick_proba,wheel_shake_proba
0,0.337219,0.344227,0.353973,0.314100,0.346557,0.322595,0.336272,0.352935,0.292123
1,0.479751,0.442886,0.292707,0.333411,0.280346,0.284728,0.328361,0.325581,0.232230
2,0.348240,0.342305,0.317633,0.329424,0.307271,0.362316,0.348468,0.359000,0.285343
3,0.339210,0.347235,0.317740,0.374427,0.301501,0.341304,0.311298,0.377495,0.289790
4,0.359307,0.372359,0.335765,0.314143,0.331229,0.346983,0.335586,0.364043,0.240584
...,...,...,...,...,...,...,...,...,...
580,0.369099,0.341373,0.305871,0.342375,0.330898,0.326672,0.325671,0.366269,0.291773
581,0.339598,0.333036,0.288180,0.336054,0.308584,0.371604,0.303122,0.416257,0.303565
582,0.331052,0.395528,0.270822,0.383494,0.339379,0.317954,0.315611,0.353631,0.292527
583,0.324123,0.358697,0.324760,0.341667,0.334138,0.321043,0.351365,0.364555,0.279653


In [34]:
fails = tot_preds.columns
fails

Index(['another_bug_proba', 'break_bug_proba', 'electro_bug_proba',
       'engine_check_proba', 'engine_fuel_proba', 'engine_ignition_proba',
       'engine_overheat_proba', 'gear_stick_proba', 'wheel_shake_proba'],
      dtype='object')

In [47]:
pd.DataFrame(tot_preds.idxmax(axis=1).str.replace('_proba', ''), columns=['soft_preds']).to_csv('sub_total_preds.csv', index=False)

In [49]:
rank = pd.DataFrame({'id': [0, 1, 2, 3, 4], 'proba': [.374540, .950714, .731994, .598658, .156019]})
rank

,id,proba
0,0,0.374540
1,1,0.950714
2,2,0.731994
3,3,0.598658
4,4,0.156019


In [89]:
rank_new = rank.sort_values(by='proba').reset_index(drop=True).reset_index().rename(columns={'index': 'normalized_proba'})
rank_new['normalized_proba'] = rank_new['normalized_proba'].add(1).div(len(rank_new))
rank_new.set_index('id').sort_index().reset_index()['normalized_proba']

0    0.4
1    1.0
2    0.8
3    0.6
4    0.2
Name: normalized_proba, dtype: float64

In [94]:
rank['proba'].rank(pct=1)

0    0.4
1    1.0
2    0.8
3    0.6
4    0.2
Name: proba, dtype: float64

In [97]:
rank2 = pd.DataFrame({'id': [0, 1, 2, 3, 4],
              'proba1': [.374540, .950714, .731994, .598658, .156019],
              'proba2': [.374540, .950714, .731994, .598658, .156019]})
rank2

,id,proba1,proba2
0,0,0.374540,0.374540
1,1,0.950714,0.950714
2,2,0.731994,0.731994
3,3,0.598658,0.598658
4,4,0.156019,0.156019


In [101]:
rank2[['proba1','proba2']].rank(pct=1).mean(axis=1)

0    0.4
1    1.0
2    0.8
3    0.6
4    0.2
dtype: float64

In [95]:
pd.DataFrame.rank?

Signature:
pd.DataFrame.rank(
    self,
    axis: 'Axis' = 0,
    method: "Literal['average', 'min', 'max', 'first', 'dense']" = 'average',
    numeric_only: 'bool_t' = False,
    na_option: "Literal['keep', 'top', 'bottom']" = 'keep',
    ascending: 'bool_t' = True,
    pct: 'bool_t' = False,
) -> 'Self'
Docstring:
Compute numerical data ranks (1 through n) along axis.

By default, equal values are assigned a rank that is the average of the
ranks of those values.

Parameters
----------
axis : {0 or 'index', 1 or 'columns'}, default 0
    Index to direct ranking.
    For `Series` this parameter is unused and defaults to 0.
method : {'average', 'min', 'max', 'first', 'dense'}, default 'average'
    How to rank the group of records that have the same value (i.e. ties):

    * average: average rank of the group
    * min: lowest rank in the group
    * max: highest rank in the group
    * first: ranks assigned in order they appear in the array
    * dense: like 'min', but rank always incr